# Import libraries

In [1]:
import os, glob, csv, sys, re
from os import path
from urllib.parse import urlparse
import pandas as pd
from bs4 import BeautifulSoup
from tld import get_tld

# Define directories

In [2]:
userhome = os.path.expanduser('~')
analyze_dir = userhome + r'/mining-forum/analyze/'
summ_dir = userhome + r'/mining-forum/analyze/csv_files/summarize/'

# Open cleaned dataset & drop duplicates

In [3]:
df = pd.read_csv(summ_dir + 'all_joined_links.csv')
df = df[['file_number']]
data_file = df.drop_duplicates(subset=None, keep='first', inplace=False)
data_file

,file_number
0,32
1,42
4,59
5,61
6,73
7,104
8,116
16,135
19,157
21,168


In [4]:
data_file.iloc[7][0]

135

# Get links from files

In [5]:
url_data = []
for file_num in range(0,len(data_file)):
    sys.stdout.write('\rExtracting file number: %i' %(file_num+1) + ' / %i' %(len(data_file)))
    sys.stdout.flush()

    num = data_file.iloc[file_num][0]
    if num <= 100000:
        number = '01'
    elif num <= 200000:
        number = '02'
    elif num <= 300000:
        number = '03'
    elif num <= 400000:
        number = '04'
    elif num <= 500000:
        number = '05'
    elif num <= 600000:
        number = '06'
    elif num <= 700000:
        number = '07'
    elif num <= 800000:
        number = '08'
    elif num <= 900000:
        number = '09'
    elif num <= 1000000:
        number = '10'
    else:
        number = '11'

    dataset = userhome + r'/mining-forum/data/forumfiles_' + number + '/'

    file = dataset + str(num)
    f = open(file)
    f = f.read()

    try:
        soup = BeautifulSoup(f, 'html.parser')
        topic = soup.find('a', attrs={'class':'MsgSubText'})
        for title in topic.stripped_strings:
            title = title

        msid = []
        soup = BeautifulSoup(f, 'html.parser')
        for message_id in soup.find_all('td', attrs={'class':'MsgR1 vt al MsgSubText'}):
            for mid in message_id.find_all('span', attrs={'class':'SmallText'}):
                for msgid in mid.find_all('a'):
                    msid.append(msgid.text)
        msid = msid[0]

        data = [num, msid, title]
        soup = BeautifulSoup(f, 'html.parser')
        for userlink in soup.find_all('td', attrs={'class':'MsgSpacer'}):
            for user in userlink.find_all('td', attrs={'class':'msgud'}):
                for usr in user.find_all('a'):
                    username = usr.text
                    if username == '':
                        username = 'Eclipse User'

                    for link in userlink.find_all('span', attrs={'class':'MsgBodyText'}):
                        for atag in link.find_all('a'):
                            ss = atag.text.replace(' ','')
                            parser = []
                            try:
                                link = re.search('^https?:\/\/.*[\r\n]*[^.\\,:;]', ss)
                                link = link.group()
                                domain = urlparse(link)
                                tld_dt = get_tld(link)
                                parser = [username, link, domain.netloc, tld_dt]
                                data.insert(len(data),parser)
                            except:
                                pass

        n = 3
        link_name = []
        for i in range (0,len(data)-3):
            if data[n][1] not in link_name:
                url_data.append([data[0], data[1], data[2], data[n][0], data[n][1], data[n][2], data[n][3]])
                link_name.append(data[n][1])
            else:
                pass
            n = n + 1
    
    except:
        pass

print ('\nFiles extraction are finished')

Extracting file number: 36206 / 36206
Files extraction are finished


In [6]:
len(url_data)

72097

In [7]:
url_data

[[32,
  'message #3',
  'Welcome to the Business Intelligence and Reporting Tools Newsgroup!',
  'Paul Clenahan',
  'http://www.eclipse.org/proposals/eclipse-birt/index.html',
  'www.eclipse.org',
  'org'],
 [42,
  'message #4',
  'SVG support in charting engine',
  'Eclipse User',
  'http://www.oasis-open.org/committees/office/faq.php',
  'www.oasis-open.org',
  'org'],
 [42,
  'message #4',
  'SVG support in charting engine',
  'Eclipse User',
  'http://www.openoffice.org/',
  'www.openoffice.org',
  'org'],
 [42,
  'message #4',
  'SVG support in charting engine',
  'Eclipse User',
  'http://news.zdnet.com/2100-3513_22-5979150.html',
  'news.zdnet.com',
  'com'],
 [59,
  'message #7',
  'Question',
  'Jeff Duska',
  'http://www.eclipse.org/org/documents/Eclipse%20Development%20Process%202003_11_09%20FINAL.pdf',
  'www.eclipse.org',
  'org'],
 [61,
  'message #9',
  'Inital thoughts on proposal',
  'Ed Burnette',
  'http://www.actuate.com/corporate/corporateresources.asp?ArticleId=72

In [8]:
with open (summ_dir + 'all_links_with_authors.csv', 'w') as csvfile:
    writers = csv.writer(csvfile, delimiter = ",")
    header = ['file_number', 'message_id', 'topic', 'author', 'link', 'domain', 'top_level_domain']
    writers.writerow(header)
    for item in url_data:
        writers.writerow(item)
print ('CSV file has been created')

CSV file has been created


In [9]:
df_link = pd.read_csv(summ_dir + 'all_links_with_authors.csv')
df_link

,file_number,message_id,topic,author,link,domain,top_level_domain
0,32,message #3,Welcome to the Business Intelligence and Repor...,Paul Clenahan,http://www.eclipse.org/proposals/eclipse-birt/...,www.eclipse.org,org
1,42,message #4,SVG support in charting engine,Eclipse User,http://www.oasis-open.org/committees/office/fa...,www.oasis-open.org,org
2,42,message #4,SVG support in charting engine,Eclipse User,http://www.openoffice.org/,www.openoffice.org,org
3,42,message #4,SVG support in charting engine,Eclipse User,http://news.zdnet.com/2100-3513_22-5979150.html,news.zdnet.com,com
4,59,message #7,Question,Jeff Duska,http://www.eclipse.org/org/documents/Eclipse%2...,www.eclipse.org,org
5,61,message #9,Inital thoughts on proposal,Ed Burnette,http://www.actuate.com/corporate/corporatereso...,www.actuate.com,com
6,73,message #11,Introducing Certive,Mohsin Beg,http://www.certive.com,www.certive.com,com
7,104,message #23,Some thoughts,Jody Garnett,http://udig.refractions.net/,udig.refractions.net,net
8,116,message #28,Geographical Charts,Eclipse User,http://www.visualmining.com/ncs/projects/Examp...,www.visualmining.com,com
9,116,message #28,Geographical Charts,Wenfeng Li,http://www.visualmining.com/ncs/projects/Examp...,www.visualmining.com,com


# Remove links if the author is Eclipse Webmaster

In [11]:
cols = ['file_number','message_id','topic','author','link','domain','top_level_domain']
new_df_link = df_link[cols][df_link[cols]['author'] != 'Eclipse Webmaster']
new_df_link

,file_number,message_id,topic,author,link,domain,top_level_domain
0,32,message #3,Welcome to the Business Intelligence and Repor...,Paul Clenahan,http://www.eclipse.org/proposals/eclipse-birt/...,www.eclipse.org,org
1,42,message #4,SVG support in charting engine,Eclipse User,http://www.oasis-open.org/committees/office/fa...,www.oasis-open.org,org
2,42,message #4,SVG support in charting engine,Eclipse User,http://www.openoffice.org/,www.openoffice.org,org
3,42,message #4,SVG support in charting engine,Eclipse User,http://news.zdnet.com/2100-3513_22-5979150.html,news.zdnet.com,com
4,59,message #7,Question,Jeff Duska,http://www.eclipse.org/org/documents/Eclipse%2...,www.eclipse.org,org
5,61,message #9,Inital thoughts on proposal,Ed Burnette,http://www.actuate.com/corporate/corporatereso...,www.actuate.com,com
6,73,message #11,Introducing Certive,Mohsin Beg,http://www.certive.com,www.certive.com,com
7,104,message #23,Some thoughts,Jody Garnett,http://udig.refractions.net/,udig.refractions.net,net
8,116,message #28,Geographical Charts,Eclipse User,http://www.visualmining.com/ncs/projects/Examp...,www.visualmining.com,com
9,116,message #28,Geographical Charts,Wenfeng Li,http://www.visualmining.com/ncs/projects/Examp...,www.visualmining.com,com


In [13]:
new_df_link.to_csv(summ_dir + 'all_links_without_bot_authors.csv', index=False)
print ("CSV file has been created")

CSV file has been created
